# 对算法的运行结果记录进行汇总，输出最终结果


## colab setting
- extra code for execute in colab
    - import module
    - modify path

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    pathname = '/mcapplus/' # the name of sync folder
    if os.path.exists('/content/drive/MyDrive'+pathname):
        return '/content/drive/MyDrive/' + pathname #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
# 当前路径
import os
path_sync = os.getcwd()

## small system result
- 小系统结果汇总


### component condition is less
组件不够情况

In [14]:
import numpy as np
import pandas as pd
from codehub_mcapplus import Common,Project,Case
import math
import copy
import time
import os
from collections import defaultdict

首先读取各个系统的pandas,以系统名字为key保存为字典

In [37]:
pattern = ['g','f']
nk = [[2,7],[3,7],[2,8],[3,8],[3,9],[4,9],[3,10]]
# ,[3,10],[4,10]
condition = ['less']
namelist_small = []

for n in nk:
    for p in pattern:
        for c in condition:
            namelist_small.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
# 路径

path_small = path_sync + '//result//now//smallsystem//'
# 文件读取
df_dict = defaultdict(list)
Seednum = 50
for name in namelist_small:
    filepath = path_small + name + '_seed_'+ str(Seednum) + '.csv'
    df = pd.read_csv(filepath,index_col=0,header=None)
    df.columns = pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','enumerationprotime','enumerationtime']
    ])
    df = df.loc[:,(slice(None),['ssr','processtime','enumerationprotime'])]
    df_dict[name] = df

In [38]:
# df_dict[namelist_small[0]]

In [39]:
summary = pd.DataFrame(index = namelist_small,
                      columns= pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['mssr','time1','time2']
    ]))
# summary

In [40]:
for name in namelist_small:
    df = df_dict[name].sum(axis=0)
    summary.loc[name,:] = np.array(df)
summary.loc[:,(slice(None),'mssr')] = summary.loc[:,(slice(None),'mssr')] / Seednum

summary.loc[:,(slice(None),'time1')] = summary.loc[:,(slice(None),'time1')] / Seednum
summary.loc[:,(slice(None),'time2')] = summary.loc[:,(slice(None),'time2')] / Seednum
summary

low                           high                       \
                 mssr     time1      time2      mssr     time1      time2   
g_2_7_less   0.992887  0.208437  0.0590625  0.951671  0.210625      0.055   
f_2_7_less   0.986575  0.203125   0.053125  0.969293  0.195625  0.0546875   
g_3_7_less   0.956412  0.179063  0.0534375  0.936875  0.215312  0.0528125   
f_3_7_less   0.971316  0.221562   0.058125  0.873093  0.211562   0.058125   
g_2_8_less   0.999116   0.27375   0.427812  0.995954      0.25       0.43   
f_2_8_less   0.969481    0.2625   0.440312  0.983581  0.259375   0.434063   
g_3_8_less   0.997592    0.2475    0.44125  0.988288  0.237187   0.427812   
f_3_8_less   0.983174  0.234063    0.43625  0.948581   0.21875      0.415   
g_3_9_less   0.999394  0.282813    3.62438  0.998168  0.299375    3.74375   
f_3_9_less   0.994394      0.34    3.80906  0.990835  0.317188        3.7   
g_4_9_less   0.967181  0.269688    3.61531  0.992013    0.2925     3.5975   
f_4_9_less   0.995387  0.299687    3.61313  0.905475    0.2475    3.61469   
g_3_10_less  0.988877  0.324688    34.8884  0.996169   0.37875    35.4894   
f_3_10_less  0.990516  0.400313    36.9712  0.919114   0.35625    38.0706   

            arbitrary                       
                 mssr     time1      time2  
g_2_7_less   0.992955  0.211562  0.0559375  
f_2_7_less   0.990572     0.205  0.0534375  
g_3_7_less   0.967322  0.196875  0.0540625  
f_3_7_less   0.967133   0.21875   0.054375  
g_2_8_less   0.994103    0.2525    0.41375  
f_2_8_less   0.978234  0.263125   0.427187  
g_3_8_less   0.994232   0.24125     0.4375  
f_3_8_less   0.956977  0.230313   0.407187  
g_3_9_less   0.997689   0.31875    3.83719  
f_3_9_less   0.985247  0.313437    3.59344  
g_4_9_less   0.987516  0.301563    3.59219  
f_4_9_less    0.98811  0.320312    3.55469  
g_3_10_less  0.997481  0.421875    35.7437  
f_3_10_less  0.981714  0.433125    37.9297

In [41]:
from collections import defaultdict
code_dict = defaultdict(list)
for name in namelist_small:
    pattern,k,n,_ = name.split('_')
    num = nk.index([int(k),int(n)]) + 1
    code = pattern + str(num)
    code_dict[name] = code
# code_dict

In [42]:
code_index = []
for name in summary.index: 
    code_index.append(code_dict[name])
summary.index = code_index
summary

low                           high                      arbitrary  \
        mssr     time1      time2      mssr     time1      time2      mssr   
g1  0.992887  0.208437  0.0590625  0.951671  0.210625      0.055  0.992955   
f1  0.986575  0.203125   0.053125  0.969293  0.195625  0.0546875  0.990572   
g2  0.956412  0.179063  0.0534375  0.936875  0.215312  0.0528125  0.967322   
f2  0.971316  0.221562   0.058125  0.873093  0.211562   0.058125  0.967133   
g3  0.999116   0.27375   0.427812  0.995954      0.25       0.43  0.994103   
f3  0.969481    0.2625   0.440312  0.983581  0.259375   0.434063  0.978234   
g4  0.997592    0.2475    0.44125  0.988288  0.237187   0.427812  0.994232   
f4  0.983174  0.234063    0.43625  0.948581   0.21875      0.415  0.956977   
g5  0.999394  0.282813    3.62438  0.998168  0.299375    3.74375  0.997689   
f5  0.994394      0.34    3.80906  0.990835  0.317188        3.7  0.985247   
g6  0.967181  0.269688    3.61531  0.992013    0.2925     3.5975  0.987516   
f6  0.995387  0.299687    3.61313  0.905475    0.2475    3.61469   0.98811   
g7  0.988877  0.324688    34.8884  0.996169   0.37875    35.4894  0.997481   
f7  0.990516  0.400313    36.9712  0.919114   0.35625    38.0706  0.981714   

                         
       time1      time2  
g1  0.211562  0.0559375  
f1     0.205  0.0534375  
g2  0.196875  0.0540625  
f2   0.21875   0.054375  
g3    0.2525    0.41375  
f3  0.263125   0.427187  
g4   0.24125     0.4375  
f4  0.230313   0.407187  
g5   0.31875    3.83719  
f5  0.313437    3.59344  
g6  0.301563    3.59219  
f6  0.320312    3.55469  
g7  0.421875    35.7437  
f7  0.433125    37.9297

### component condition is more
组件不完全够情况

In [2]:
import numpy as np
import pandas as pd
from codehub_mcapplus import Common,Project,Case
import math
import copy
import time
import os
from collections import defaultdict

In [3]:
import os
path_sync = os.getcwd()

In [4]:
pattern = ['g','f']
nk = [[2,7],[3,7],[2,8],[3,8]]
# ,[4,9],[3,10]
# ,[3,10],[4,10]
condition = ['more']
namelist_small = []

for n in nk:
    for p in pattern:
        for c in condition:
            namelist_small.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
# 路径

path_small = path_sync + '//result//now//smallsystem//'
# 文件读取
df_dict = defaultdict(list)
Seednum = 50
for name in namelist_small:
    filepath = path_small + name + '_seed_'+ str(Seednum) + '.csv'
    df = pd.read_csv(filepath,index_col=0,header=None)
    df.columns = pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','enumerationprotime','enumerationtime']
    ])
    df = df.loc[:,(slice(None),['ssr','processtime','enumerationprotime'])]
    df_dict[name] = df

In [5]:
summary = pd.DataFrame(index = namelist_small,
                      columns= pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['mssr','time1','time2']
    ]))
summary

low             high             arbitrary            
           mssr time1 time2 mssr time1 time2      mssr time1 time2
g_2_7_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_2_7_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_3_7_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_3_7_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_2_8_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_2_8_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_3_8_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_3_8_more  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN

In [6]:
for name in namelist_small:
    df = df_dict[name].sum(axis=0)
    summary.loc[name,:] = np.array(df)
summary.loc[:,(slice(None),'mssr')] = summary.loc[:,(slice(None),'mssr')] / Seednum
summary.loc[:,(slice(None),'time1')] = summary.loc[:,(slice(None),'time1')] / Seednum
summary.loc[:,(slice(None),'time2')] = summary.loc[:,(slice(None),'time2')] / Seednum
summary

low                          high                      \
                mssr     time1     time2      mssr     time1     time2   
g_2_7_more  0.964654  0.319063      0.48  0.855438  0.312812  0.472813   
f_2_7_more  0.916198  0.360625   0.47875  0.812094  0.355625  0.467813   
g_3_7_more  0.944737  0.279687   0.47125  0.836713  0.340313  0.473438   
f_3_7_more  0.949924  0.290312  0.471562  0.871437  0.275313  0.469062   
g_2_8_more  0.963923  0.363438   4.01438  0.826605  0.349062   4.05312   
f_2_8_more  0.890105  0.342813   4.01594  0.759219  0.349687   4.05844   
g_3_8_more  0.953013   0.29875   3.94062  0.790431    0.3175   4.02125   
f_3_8_more  0.924149   0.31125   4.01187  0.820997  0.323437   3.92313   

           arbitrary                      
                mssr     time1     time2  
g_2_7_more  0.955223  0.341562  0.482812  
f_2_7_more   0.86406  0.345625  0.474375  
g_3_7_more  0.887341  0.287813    0.4725  
f_3_7_more  0.928409      0.28  0.458437  
g_2_8_more  0.961159   0.34625     4.005  
f_2_8_more  0.857254  0.356875    4.0575  
g_3_8_more  0.886915  0.323437     4.015  
f_3_8_more  0.892337  0.351562   4.04844

In [7]:
from collections import defaultdict
code_dict = defaultdict(list)
for name in namelist_small:
    pattern,k,n,_ = name.split('_')
    num = nk.index([int(k),int(n)]) + 1
    code = pattern + str(num)
    code_dict[name] = code
# code_dict
code_index = []
for name in summary.index: 
    code_index.append(code_dict[name])
summary.index = code_index
summary

low                          high                     arbitrary  \
        mssr     time1     time2      mssr     time1     time2      mssr   
g1  0.964654  0.319063      0.48  0.855438  0.312812  0.472813  0.955223   
f1  0.916198  0.360625   0.47875  0.812094  0.355625  0.467813   0.86406   
g2  0.944737  0.279687   0.47125  0.836713  0.340313  0.473438  0.887341   
f2  0.949924  0.290312  0.471562  0.871437  0.275313  0.469062  0.928409   
g3  0.963923  0.363438   4.01438  0.826605  0.349062   4.05312  0.961159   
f3  0.890105  0.342813   4.01594  0.759219  0.349687   4.05844  0.857254   
g4  0.953013   0.29875   3.94062  0.790431    0.3175   4.02125  0.886915   
f4  0.924149   0.31125   4.01187  0.820997  0.323437   3.92313  0.892337   

                        
       time1     time2  
g1  0.341562  0.482812  
f1  0.345625  0.474375  
g2  0.287813    0.4725  
f2      0.28  0.458437  
g3   0.34625     4.005  
f3  0.356875    4.0575  
g4  0.323437     4.015  
f4  0.351562   4.04844

- 大系统结果汇总

In [ ]:
pattern = ['g','f']
nk = [[4,20],[5,20],[10,20],[12,30],[15,30],[20,30],[22,40],[25,40],[30,40],[32,50],[35,50],[40,50]]
condition = ['less']
namelist_large = []
for p in pattern:
    for n in nk:
        for c in condition:
            namelist_large.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
print(namelist_large)
# 路径
path_large = '.\\result\\now\\largesystem\\'
# 文件读取
df_dict = defaultdict(list)
Seednum = 10
for name in namelist_large:
    filepath = path_large + name + '_seed_'+ str(Seednum) + '.csv'
    df = pd.read_csv(filepath,index_col=0,header=None)
    df.columns = pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','enumerationprotime','enumerationtime']
    ])
    df = df.loc[:,(slice(None),['ssr','processtime','enumerationprotime'])]
    df_dict[name] = df

In [ ]:
df_dict[namelist_large[0]]

In [ ]:
summary = pd.DataFrame(index = namelist_large,
                      columns= pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['mssr','time1','time2']
    ]))
summary

In [ ]:
for name in namelist_large:
    df = df_dict[name].sum(axis=0)
    summary.loc[name,:] = np.array(df)
summary.loc[:,(slice(None),'mssr')] = summary.loc[:,(slice(None),'mssr')] / Seednum
summary